In [1]:
import ECGDataset 
import Models 
import Net
from train_test_validat import *
from self_attention import *
import matplotlib.pyplot as plt
import ecg_plot
import cam
import ECGplot
import ECGHandle
import torch
import torch.utils.data as Data
import sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
import random
import pandas as pd
from tqdm import tqdm

import time
import math
import os
import gc
from torch.utils.tensorboard import SummaryWriter


def seed_torch(seed=2023):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False 
	torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.enabled = False

In [2]:
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = "cpu"

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
seed_torch(2023)

In [5]:
data_root = '/workspace/data/Preprocess_HTN/datas_/'
ALL_data = pd.read_csv(data_root+'/All_data_handled_ID_range_age_IDimputate.csv',low_memory=False)
ALL_data = ECGHandle.change_label(ALL_data)
ALL_data = ECGHandle.filter_ID(ALL_data)
ALL_data = ECGHandle.filter_QC(ALL_data)
ALL_data = ECGHandle.filter_ages(ALL_data,18)
ALL_data = ECGHandle.filter_departmentORlabel(ALL_data,'外科')
ALL_data = ECGHandle.correct_label(ALL_data)
ALL_data = ECGHandle.correct_age(ALL_data)
ALL_data = ECGHandle.filter_diagnose(ALL_data,'起搏')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'房颤')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'左束支传导阻滞')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'左前分支阻滞')
# ALL_data = ECGHandle.filter_diagnose(ALL_data,'阻滞')
# ALL_data = ECGHandle.remove_duplicated(ALL_data)
ALL_data = ALL_data.rename(columns={'住院号':'ID','年龄':'age','性别':'gender','姓名':'name'}) 
ALL_data_buffer = ALL_data.copy()
seed_torch(2023)
ALL_data_buffer = ALL_data_buffer.sample(frac=1).reset_index(drop=True) #打乱顺序
# all_dataset = ECGHandle.ECG_Dataset(data_root,ALL_data_buffer,preprocess = True)
####################################################################随机选取test
test_df,tv_df = Pair_ID(ALL_data,0.2,Range_max=15,pair_num=1)
test_dataset = ECGHandle.ECG_Dataset(data_root,test_df,preprocess = True)


Models_path = 'model/20230302_080241/20230302_080241/parameter_EarlyStoping_3.pt'
save_root = Models_path[:-3]+'/'
layervalue_root = save_root+'/layervalue/'





            orginal   removed diagnose NaN
   nums      200082          199997       
              HTN             NHTN        
   nums       3273           196724       


            orginal      removed ID NaN   
   nums      199997          199995       
              HTN             NHTN        
   nums       3273           196722       


            orginal            QC         
   nums      199995          72845        
              HTN             NHTN        
   nums       1497           71348        


            orginal      filtered ages    
   nums      72845           69819        
              HTN             NHTN        
   nums       1477           68342        


     reset num:       10  
  ERR labels num:     308 
            orginal      correct label    
   nums      69819           69819        
              HTN             NHTN        
   nums       1794           68025        


            orginal   filtered department 
   nums      69819           15

In [4]:
data_root = '/workspace/data/Preprocess_HTN/datas_/'
ALL_data = pd.read_csv(data_root+'/All_data_handled_ID_range_age_IDimputate.csv',low_memory=False)
ALL_data = ECGHandle.change_label(ALL_data)
ALL_data = ECGHandle.filter_ID(ALL_data)
ALL_data = ECGHandle.filter_QC(ALL_data)
ALL_data = ECGHandle.filter_ages(ALL_data,18)
ALL_data = ECGHandle.filter_departmentORlabel(ALL_data,'外科')



     reset num:       10  
  ERR labels num:     308 
            orginal      correct label    
   nums      69819           69819        
              HTN             NHTN        
   nums       1794           68025        




KeyboardInterrupt: 

In [ ]:
# 定义获取梯度的函数
fmap_block = list()
grad_block = list()

# 获取反向的传播图
def backward_hook(module, grad_in, grad_out):
    # print('backward_hook ',len(grad_out),grad_out[0].shape)
    grad_block.append(grad_out[0].clone().cpu().detach())

# 定义获取特征图的函数
def farward_hook(module, input, output):
    # print('farward_hook ',output.shape)
    fmap_block.append(output.clone().cpu().detach())

In [ ]:
def scaled_layer_cam(x,gamma:float = 2.):
    x_max = x.max()
    x_scaled = np.tanh(((gamma*x)/x_max))
    return x_scaled

In [ ]:
def scaled_layer_cam_max(cam):
    (cam_min, cam_max) = (cam.min(), cam.max())
    norm_cam = (cam - cam_min) / (((cam_max - cam_min) + 1e-08))
    return norm_cam

In [ ]:
Models_path = '/workspace/data/Interpretable_HTN/model/20230307_144157/20230307_144157/BestF1_0.pt'

In [ ]:
NET = [Net.MLBFNet_GUR_o(True,True,True,2,Dropout_rate=0.3), ] # type: ignore
criterion = torch.nn.CrossEntropyLoss()
testmodel = NET[0].to(DEVICE)
testmodel.load_state_dict(torch.load(Models_path))
test_dataloader = Data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)  
y_true,y_pred,y_out,test_loss,test_acc = eval_model(test_dataloader,criterion,testmodel,DEVICE) # 验证模型

## 计算出最佳阈值，并获得该阈值下的y_pred

In [ ]:
def Find_Optimal_Cutoff(TPR, FPR, threshold):
    y = TPR - FPR
    Youden_index = np.argmax(y)  # Only the first occurrence is returned.
    optimal_threshold = threshold[Youden_index]
    point = [FPR[Youden_index], TPR[Youden_index]]
    return optimal_threshold, point
def ROC(label, y_prob):
    """
    Receiver_Operating_Characteristic, ROC
    :param label: (n, )
    :param y_prob: (n, )
    :return: fpr, tpr, roc_auc, optimal_th, optimal_point
    """
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(label, y_prob)
    roc_auc = sklearn.metrics.auc(fpr, tpr)
    optimal_th, optimal_point = Find_Optimal_Cutoff(TPR=tpr, FPR=fpr, threshold=thresholds)
    return fpr, tpr, roc_auc, optimal_th, optimal_point

In [ ]:
fpr, tpr, roc_auc, optimal_th, optimal_point = ROC(y_true,((np.array(y_out))[:,1]))
plt.figure(1)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.plot(optimal_point[0], optimal_point[1], marker='o', color='r')
plt.text(optimal_point[0], optimal_point[1], f'Threshold:{optimal_th:.2f}')
plt.title("ROC-AUC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()
CM = confusion_matrix(y_true,y_pred)
print(CM)
y_pred = ((np.array(y_out))[:,1])
y_pred[y_pred>optimal_th] =  1
y_pred[y_pred<=optimal_th] =  0

## 计算layer-CAM值

In [ ]:
save_root = Models_path[:-3]+'/'
layervalue_root = save_root+'/layervalue/'
if(not(os.path.exists(save_root))): os.mkdir(save_root)
if(not(os.path.exists(layervalue_root))): os.mkdir(layervalue_root)
lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
x_index = np.arange(0,12)
# fig, axs = plt.subplots(nrows=4, ncols=3, sharex=True,sharey=True,figsize=(45,25), constrained_layout=True,dpi =100)
for index in tqdm(range( test_dataset.__len__())):# test_dataset.__len__()
    NET = [Net.MLBFNet_GUR_o(True,True,True,2,Dropout_rate=0.3), ] # type: ignore
    testmodel = NET[0].to(DEVICE)
    testmodel.load_state_dict(torch.load(Models_path))
    testmodel.conv1.register_forward_hook(farward_hook)	#正向传播
    testmodel.conv1.register_full_backward_hook(backward_hook)#反向传播
    testmodel.conv2.register_forward_hook(farward_hook)	#正向传播
    testmodel.conv2.register_full_backward_hook(backward_hook)#反向传播
    testmodel.layers_list_2d[0].register_forward_hook(farward_hook)	#正向传播
    testmodel.layers_list_2d[0].register_full_backward_hook(backward_hook)#反向传播
    testmodel.layers_list_2d[1].register_forward_hook(farward_hook)	#正向传播
    testmodel.layers_list_2d[1].register_full_backward_hook(backward_hook)#反向传播
    testmodel.layers_list_2d[2].register_forward_hook(farward_hook)	#正向传播
    testmodel.layers_list_2d[2].register_full_backward_hook(backward_hook)#反向传播


    inputs,labels = test_dataset.__getitem__(index)
    info =test_dataset.infos.iloc[index]
    ECGfile_name = info['ECGFilename']
    testmodel.eval()
    fmap_block = list()
    grad_block = list()

    labels = labels.unsqueeze(0) # 在首位添加1维作为batchsize
    inputs = inputs.unsqueeze(0) # 在首位添加1维作为batchsize

    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)  

    outputs = testmodel(inputs)
    # print('outputs',outputs)
    _,pred = outputs.max(1)     # 求概率最大值对应的标签
    
    possibility = nn.functional.softmax(outputs,dim=-1)
    # print("labels: {}".format(labels))
    # print("predict: {}".format(pred))
    # print("possibility: {}".format(possibility))
    loss = outputs[0,int(y_pred[index])]      # 网络对应于pred的类别的输出即为loss
    loss.backward(retain_graph=True)  #retain_graph=True，目的为是为保留该过程中计算的梯度，后续G网络更新时使用

    fmap1 = (fmap_block[0][0]).to('cpu').detach().numpy()
    gradmap1 = (grad_block[4][0]).to('cpu').detach().numpy()#G是从后往前的，conv1的在后面
    layer2d_vlue1 = cam.caculate_layer_cam_vlue_2d(fmap1,gradmap1)

    fmap2 = (fmap_block[1][0]).to('cpu').detach().numpy()
    gradmap2 = (grad_block[3][0]).to('cpu').detach().numpy()
    layer2d_vlue2 = cam.caculate_layer_cam_vlue_2d(fmap2,gradmap2)

    fmap2d_1 = (fmap_block[2][0]).to('cpu').detach().numpy()
    gradmap2d_1 = (grad_block[2][0]).to('cpu').detach().numpy()
    layer2d_vlue2d_1 = cam.caculate_layer_cam_vlue_2d(fmap2d_1,gradmap2d_1)

    fmap2d_2 = (fmap_block[3][0]).to('cpu').detach().numpy()
    gradmap2d_2 = (grad_block[1][0]).to('cpu').detach().numpy()
    layer2d_vlue2d_2 = cam.caculate_layer_cam_vlue_2d(fmap2d_2,gradmap2d_2)

    fmap2d_3 = (fmap_block[4][0]).to('cpu').detach().numpy()
    gradmap2d_3 = (grad_block[0][0]).to('cpu').detach().numpy()
    layer2d_vlue2d_3 = cam.caculate_layer_cam_vlue_2d(fmap2d_3,gradmap2d_3)

    layer_vlue_mean = (scaled_layer_cam_max(layer2d_vlue1)+scaled_layer_cam_max(layer2d_vlue2)+scaled_layer_cam_max(layer2d_vlue2d_1)+scaled_layer_cam_max(layer2d_vlue2d_2)+scaled_layer_cam_max(layer2d_vlue2d_3))/5
    np.save(layervalue_root+ECGfile_name,layer_vlue_mean)
    
#     inputs,_ = test_dataset.__getitem__(index)
#     cmap = 'OrRd' 
#     colors = ECGplot.color_map(layer_vlue_mean, cmap)#相当于归一化

#     for i,ax in enumerate(axs.flat):  # type: ignore
#         plot_y = np.array(inputs[i]*5000.)#缩放
#         t = np.arange(0,5000)
#         ECGplot.plot_ECG_line(fig,ax,x = t,y= plot_y,y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i])
#         for j in t:
#             ax.axvline(x=j,alpha=0.1,color=colors[i,j]) #第i导联 第j个时间点的importance value
#     plt.savefig(save_root+'/'+info['ECGFilename']+'_'+str(labels[0].tolist())+'_'+str(pred[0].tolist())+'.png', bbox_inches='tight',dpi = 200)  
# plt.close()  

In [ ]:
def max_min(x):
    max_ = x.max()
    min_ = x.min()
    x_ = (x-min_)/(max_-min_+1e-8)
    return x_

In [ ]:
def layervalue_segement_count(start_index:int,end_index:int,layervalue:np.ndarray,segement_num:int = 10):
    len_ = end_index-start_index
    chanle_num = layervalue.shape[0]
    if(len_<segement_num):
        layervalue_sum = ((layervalue[:,start_index:end_index].sum())/(chanle_num*segement_num*1.0))
        return layervalue_sum
    step = len_//segement_num
    layervalue_sum = np.zeros(segement_num)
    for i in range(segement_num):
        start = start_index+step*i
        sum_sement = layervalue[:,start:(start+step)].sum()/(chanle_num*step*1.0)
        layervalue_sum[i] = layervalue_sum[i]+sum_sement*1. #在这一段step里面，每个时间点的平均重要程度
    return layervalue_sum

In [ ]:
index = 0
segement_number = 10 # 共6个片段，每段划分成10份
important_values = np.zeros([6,segement_number])


for index in range(test_dataset.__len__()):#
    important_value = np.zeros([6,segement_number])
    inputs,labels = test_dataset.__getitem__(index)
    if(labels != y_true[index]): 
        break
    pred = y_pred[index]
    if((pred == 0) or (labels == 0)): continue;
    info =test_dataset.infos.iloc[index]
    ECGfile_name = info['ECGFilename']
    layervalue_file = layervalue_root  + ECGfile_name +'.npy'
    waves_location_file_root = '/workspace/data/Preprocess_HTN/Waves_2023/'
    waves = pd.read_csv(waves_location_file_root+ECGfile_name+'.csv',index_col=0)
    if(waves.isnull().any().any()):
        continue
    layervalue = np.load(layervalue_file)

    for head_peak in range(1,len(waves.columns)-1):
        if(waves.loc['P'][head_peak] == waves.iloc[:,head_peak].min()): #P波不是第一个识别出来的波形
            p_index = int(waves.loc['P'][head_peak])
            q_index = int(waves.loc['Q'][head_peak])
            r_index = int(waves.loc['R'][head_peak])
            s_index = int(waves.loc['S'][head_peak])
            t_index = int(waves.loc['T'][head_peak])
            t_index_befort = int(waves.loc['T'][head_peak-1])
            if(((waves.loc['P'][head_peak+1]==None)) or((waves.loc['P'][head_peak+1] < t_index))):
                break
            else:
                p_index_next = waves.loc['P'][head_peak+1] 
            important_value[0] = important_value[0]+ layervalue_segement_count(t_index_befort,p_index,layervalue,segement_number)
            important_value[1] = important_value[1]+ layervalue_segement_count(p_index,q_index,layervalue,segement_number)   
            important_value[2] = important_value[2]+ layervalue_segement_count(q_index,r_index,layervalue,segement_number) 
            important_value[3] = important_value[3]+layervalue_segement_count(r_index,s_index,layervalue,segement_number)
            important_value[4] = important_value[4]+ layervalue_segement_count(s_index,t_index,layervalue,segement_number) 
            important_value[5] = important_value[5]+ layervalue_segement_count(t_index,p_index_next,layervalue,segement_number) 
        else:
            break
    important_value = max_min(important_value)
    important_values = important_values + important_value
X_lable = ['0','T','P','Q','R','S','T','P','']
fig,ax = plt.subplots(1,1)
ax.plot(np.arange(important_values.flatten().__len__()), max_min(important_values.flatten()), lw=1,color='k')
ax.axvline(x=0,ls='dashed',lw=0.7,color='r')
ax.axvline(x=10,ls='dashed',lw=0.7,color='g')
ax.axvline(x=20,ls='dashed',lw=0.7,color='b')
ax.axvline(x=30,ls='dashed',lw=0.7,color='c')
ax.axvline(x=40,ls='dashed',lw=0.7,color='m')
ax.axvline(x=50,ls='dashed',lw=0.7,color='r')
ax.axvline(x=60,ls='dashed',lw=0.7,color='g')
ax.set_xlim(-5,65)
ax.set_xticklabels(X_lable)
plt.savefig(save_root+'/HTN.png',dpi = 300)

In [ ]:
index = 0
segement_number = 10
important_values = np.zeros([6,segement_number])


for index in range(test_dataset.__len__()):#
    important_value = np.zeros([6,segement_number])
    inputs,labels = test_dataset.__getitem__(index)
    if(labels != y_true[index]): 
        break
    pred = y_pred[index]
    if(pred == 1 or (labels == 1)): continue;
    info =test_dataset.infos.iloc[index]
    ECGfile_name = info['ECGFilename']
    layervalue_file = layervalue_root  + ECGfile_name +'.npy'
    waves_location_file_root = '/workspace/data/Preprocess_HTN/Waves_2023/'
    waves = pd.read_csv(waves_location_file_root+ECGfile_name+'.csv',index_col=0)
    if(waves.isnull().any().any()):
        continue
    layervalue = np.load(layervalue_file)

    for head_peak in range(1,len(waves.columns)-1):
        if(waves.loc['P'][head_peak] == waves.iloc[:,head_peak].min()): #P波不是第一个识别出来的波形
            p_index = int(waves.loc['P'][head_peak])
            q_index = int(waves.loc['Q'][head_peak])
            r_index = int(waves.loc['R'][head_peak])
            s_index = int(waves.loc['S'][head_peak])
            t_index = int(waves.loc['T'][head_peak])
            t_index_befort = int(waves.loc['T'][head_peak-1])
            if(((waves.loc['P'][head_peak+1]==None)) or((waves.loc['P'][head_peak+1] < t_index))):
                break
            else:
                p_index_next = waves.loc['P'][head_peak+1] 
            important_value[0] = important_value[0]+ layervalue_segement_count(t_index_befort,p_index,layervalue,segement_number)
            important_value[1] = important_value[1]+ layervalue_segement_count(p_index,q_index,layervalue,segement_number)   
            important_value[2] = important_value[2]+ layervalue_segement_count(q_index,r_index,layervalue,segement_number) 
            important_value[3] = important_value[3]+layervalue_segement_count(r_index,s_index,layervalue,segement_number)
            important_value[4] = important_value[4]+ layervalue_segement_count(s_index,t_index,layervalue,segement_number) 
            important_value[5] = important_value[5]+ layervalue_segement_count(t_index,p_index_next,layervalue,segement_number) 
        else:
            break
    important_value = max_min(important_value)
    important_values = important_values + important_value
X_lable = ['0','T','P','Q','R','S','T','P','']
fig,ax = plt.subplots(1,1)
ax.plot(np.arange(important_values.flatten().__len__()), max_min(important_values.flatten()), lw=1,color='k')
ax.axvline(x=0,ls='dashed',lw=0.7,color='r')
ax.axvline(x=10,ls='dashed',lw=0.7,color='g')
ax.axvline(x=20,ls='dashed',lw=0.7,color='b')
ax.axvline(x=30,ls='dashed',lw=0.7,color='c')
ax.axvline(x=40,ls='dashed',lw=0.7,color='m')
ax.axvline(x=50,ls='dashed',lw=0.7,color='r')
ax.axvline(x=60,ls='dashed',lw=0.7,color='g')
ax.set_xlim(-5,65)
ax.set_xticklabels(X_lable)
plt.savefig(save_root+'/NHTN.png',dpi = 300)

In [ ]:
index = 0
segement_number = 10
ECG_values = np.zeros([6,segement_number])


for index in range(test_dataset.__len__()):#
    ECG_value = np.zeros([6,segement_number])
    inputs,labels = test_dataset.__getitem__(index)
    info =test_dataset.infos.iloc[index]
    ECGfile_name = info['ECGFilename']
    layervalue_file = layervalue_root  + ECGfile_name +'.npy'
    waves_location_file_root = '/workspace/data/Preprocess_HTN/Waves_2023/'
    waves = pd.read_csv(waves_location_file_root+ECGfile_name+'.csv',index_col=0)
    if(waves.isnull().any().any()):
        continue
    # layervalue = np.load(layervalue_file)
    layervalue = np.array(inputs)
    for head_peak in range(1,len(waves.columns)-1):
        if(waves.loc['P'][head_peak] == waves.iloc[:,head_peak].min()): #P波不是第一个识别出来的波形
            p_index = int(waves.loc['P'][head_peak])
            q_index = int(waves.loc['Q'][head_peak])
            r_index = int(waves.loc['R'][head_peak])
            s_index = int(waves.loc['S'][head_peak])
            t_index = int(waves.loc['T'][head_peak])
            t_index_befort = int(waves.loc['T'][head_peak-1])
            if(((waves.loc['P'][head_peak+1]==None)) or((waves.loc['P'][head_peak+1] < t_index))):
                break
            else:
                p_index_next = waves.loc['P'][head_peak+1] 
            ECG_value[0] = ECG_value[0]+ layervalue_segement_count(t_index_befort,p_index,layervalue,segement_number)
            ECG_value[1] = ECG_value[1]+ layervalue_segement_count(p_index,q_index,layervalue,segement_number)   
            ECG_value[2] = ECG_value[2]+ layervalue_segement_count(q_index,r_index,layervalue,segement_number) 
            ECG_value[3] = ECG_value[3]+layervalue_segement_count(r_index,s_index,layervalue,segement_number)
            ECG_value[4] = ECG_value[4]+ layervalue_segement_count(s_index,t_index,layervalue,segement_number) 
            ECG_value[5] = ECG_value[5]+ layervalue_segement_count(t_index,p_index_next,layervalue,segement_number) 
        else:
            break
    # ECG_value = max_min(ECG_value)
    ECG_values = ECG_values + ECG_value
X_lable = ['0','T','P','Q','R','S','T','P','']
fig,ax = plt.subplots(1,1)
ax.plot(np.arange(ECG_values.flatten().__len__()), max_min(ECG_values.flatten()), lw=1,color='k')
ax.axvline(x=0,ls='dashed',lw=0.7,color='r')
ax.axvline(x=10,ls='dashed',lw=0.7,color='g')
ax.axvline(x=20,ls='dashed',lw=0.7,color='b')
ax.axvline(x=30,ls='dashed',lw=0.7,color='c')
ax.axvline(x=40,ls='dashed',lw=0.7,color='m')
ax.axvline(x=50,ls='dashed',lw=0.7,color='r')
ax.axvline(x=60,ls='dashed',lw=0.7,color='g')
ax.set_xlim(-5,65)
ax.set_xticklabels(X_lable)

## 获取各个导联的心拍曲线

In [ ]:
def leads_layervalue_segement_count(start_index:int,end_index:int,layervalue:np.ndarray,segement_num:int = 10):
    len_ = end_index-start_index
    chanle_num = layervalue.shape[0]
    layervalue_sum = np.zeros([chanle_num,segement_num])
    if(len_<segement_num):
        for i in range(segement_num):
           layervalue_sum[:,i] = layervalue_sum[:,i] + ((layervalue[:,start_index:end_index].sum(axis = 1))/(segement_num*1.0)) #返回平均值，广播效应
        return layervalue_sum
    step = len_//segement_num
    for i in range(segement_num):
        start = start_index+step*i
        sum_sement = layervalue[:,start:(start+step)].sum(axis = 1)/(chanle_num*step*1.0)
        layervalue_sum[:,i] = layervalue_sum[:,i]+sum_sement*1. #在这一段step里面，每个时间点的平均重要程度
    return layervalue_sum

In [ ]:
index = 0
segement_num = 10
NHTN_important_values = np.zeros([6,12,segement_num])
NHTN_ECG_values = np.zeros([6,12,segement_num])
index_count= 0
for index in range(test_dataset.__len__()):#
    
    inputs,labels = test_dataset.__getitem__(index)
    if(labels == 1): continue
    info =test_dataset.infos.iloc[index]
    ECGfile_name = info['ECGFilename']
    layervalue_file = layervalue_root  + ECGfile_name +'.npy'
    waves_location_file_root = '/workspace/data/Preprocess_HTN/Waves_2023/'
    waves = pd.read_csv(waves_location_file_root+ECGfile_name+'.csv',index_col=0)
    if(waves.isnull().any().any()):
        continue
    layervalue = np.load(layervalue_file)
    peak_count = 0
    important_value = np.zeros([6,12,segement_num])
    ECG_value = np.zeros([6,12,segement_num])
    for head_peak in range(1,len(waves.columns)-1):
        if(waves.loc['P'][head_peak] == waves.iloc[:,head_peak].min()): #P波不是第一个识别出来的波形
            p_index = int(waves.loc['P'][head_peak])
            q_index = int(waves.loc['Q'][head_peak])
            r_index = int(waves.loc['R'][head_peak])
            s_index = int(waves.loc['S'][head_peak])
            t_index = int(waves.loc['T'][head_peak])
            t_index_befort = int(waves.loc['T'][head_peak-1])
            if(((waves.loc['P'][head_peak+1]==None)) or((waves.loc['P'][head_peak+1] < t_index))):
                break
            else:
                p_index_next = waves.loc['P'][head_peak+1] 
        else:
            break    
        
        important_value[0] = important_value[0]+ leads_layervalue_segement_count(t_index_befort,p_index,layervalue,segement_num)
        important_value[1] = important_value[1]+ leads_layervalue_segement_count(p_index,q_index,layervalue,segement_num)   
        important_value[2] = important_value[2]+ leads_layervalue_segement_count(q_index,r_index,layervalue,segement_num) 
        important_value[3] = important_value[3]+leads_layervalue_segement_count(r_index,s_index,layervalue,segement_num)
        important_value[4] = important_value[4]+ leads_layervalue_segement_count(s_index,t_index,layervalue,segement_num) 
        important_value[5] = important_value[5]+ leads_layervalue_segement_count(t_index,p_index_next,layervalue,segement_num) 
        
        
        ECG_value[0] = ECG_value[0]+ leads_layervalue_segement_count(t_index_befort,p_index,layervalue,segement_num)
        ECG_value[1] = ECG_value[1]+ leads_layervalue_segement_count(p_index,q_index,layervalue,segement_num)   
        ECG_value[2] = ECG_value[2]+ leads_layervalue_segement_count(q_index,r_index,layervalue,segement_num) 
        ECG_value[3] = ECG_value[3]+leads_layervalue_segement_count(r_index,s_index,layervalue,segement_num)
        ECG_value[4] = ECG_value[4]+ leads_layervalue_segement_count(s_index,t_index,layervalue,segement_num) 
        ECG_value[5] = ECG_value[5]+ leads_layervalue_segement_count(t_index,p_index_next,layervalue,segement_num) 
        peak_count = peak_count+1
    
    ECG_value = ECG_value/peak_count
    important_value = important_value/peak_count
    
    important_value = max_min(important_value)
    NHTN_important_values = NHTN_important_values + important_value
    NHTN_ECG_values = NHTN_ECG_values +ECG_value


In [ ]:
index = 0
segement_num = 10
HTN_important_values = np.zeros([6,12,segement_num])
HTN_ECG_values = np.zeros([6,12,segement_num])
index_count= 0
for index in range(test_dataset.__len__()):#
    
    inputs,labels = test_dataset.__getitem__(index)
    if(labels == 0): continue
    info =test_dataset.infos.iloc[index]
    ECGfile_name = info['ECGFilename']
    layervalue_file = layervalue_root  + ECGfile_name +'.npy'
    waves_location_file_root = '/workspace/data/Preprocess_HTN/Waves_2023/'
    waves = pd.read_csv(waves_location_file_root+ECGfile_name+'.csv',index_col=0)
    if(waves.isnull().any().any()):
        continue
    layervalue = np.load(layervalue_file)
    peak_count = 0
    important_value = np.zeros([6,12,segement_num])
    ECG_value = np.zeros([6,12,segement_num])
    for head_peak in range(1,len(waves.columns)-1):
        if(waves.loc['P'][head_peak] == waves.iloc[:,head_peak].min()): #P波不是第一个识别出来的波形
            p_index = int(waves.loc['P'][head_peak])
            q_index = int(waves.loc['Q'][head_peak])
            r_index = int(waves.loc['R'][head_peak])
            s_index = int(waves.loc['S'][head_peak])
            t_index = int(waves.loc['T'][head_peak])
            t_index_befort = int(waves.loc['T'][head_peak-1])
            if(((waves.loc['P'][head_peak+1]==None)) or((waves.loc['P'][head_peak+1] < t_index))):
                break
            else:
                p_index_next = waves.loc['P'][head_peak+1] 
        else:
            break    
        
        important_value[0] = important_value[0]+ leads_layervalue_segement_count(t_index_befort,p_index,layervalue,segement_num)
        important_value[1] = important_value[1]+ leads_layervalue_segement_count(p_index,q_index,layervalue,segement_num)   
        important_value[2] = important_value[2]+ leads_layervalue_segement_count(q_index,r_index,layervalue,segement_num) 
        important_value[3] = important_value[3]+leads_layervalue_segement_count(r_index,s_index,layervalue,segement_num)
        important_value[4] = important_value[4]+ leads_layervalue_segement_count(s_index,t_index,layervalue,segement_num) 
        important_value[5] = important_value[5]+ leads_layervalue_segement_count(t_index,p_index_next,layervalue,segement_num) 
        
        
        ECG_value[0] = ECG_value[0]+ leads_layervalue_segement_count(t_index_befort,p_index,layervalue,segement_num)
        ECG_value[1] = ECG_value[1]+ leads_layervalue_segement_count(p_index,q_index,layervalue,segement_num)   
        ECG_value[2] = ECG_value[2]+ leads_layervalue_segement_count(q_index,r_index,layervalue,segement_num) 
        ECG_value[3] = ECG_value[3]+leads_layervalue_segement_count(r_index,s_index,layervalue,segement_num)
        ECG_value[4] = ECG_value[4]+ leads_layervalue_segement_count(s_index,t_index,layervalue,segement_num) 
        ECG_value[5] = ECG_value[5]+ leads_layervalue_segement_count(t_index,p_index_next,layervalue,segement_num) 
        peak_count = peak_count+1
    
    ECG_value = ECG_value/peak_count
    important_value = important_value/peak_count
                
    important_value = max_min(important_value)
    HTN_important_values = HTN_important_values + important_value
    HTN_ECG_values = HTN_ECG_values +ECG_value


In [ ]:
lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
X_lable = ['T','P','Q','R','S','T','P']

fig, axs = plt.subplots(nrows=3, ncols=4,sharey=True,figsize=(10,10), constrained_layout=True,dpi =100)

cmap = 'OrRd' 
colors = ECGplot.color_map(important_values, cmap)#相当于归一化

for i,ax in enumerate(axs.flat):  # type: ignore
    HTN_plot_y = np.array(HTN_ECG_values[:,i,:].flatten())
    HTN_colors_z = np.array(HTN_important_values[:,i,:].flatten())
    NHTN_plot_y = np.array(NHTN_ECG_values[:,i,:].flatten())
    NHTN_colors_z = np.array(NHTN_important_values[:,i,:].flatten())    
    t = np.arange(0,HTN_plot_y.shape[0]) #  
    ax.plot(t, max_min(HTN_colors_z), lw=1,color='r')
    ax.plot(t, max_min(NHTN_colors_z), lw=1,color='b')
    ax.axvline(x=0,ls='dashed',lw=0.7,color='k')
    ax.axvline(x=10,ls='dashed',lw=0.7,color='k')
    ax.axvline(x=20,ls='dashed',lw=0.7,color='k')
    ax.axvline(x=30,ls='dashed',lw=0.7,color='k')
    ax.axvline(x=40,ls='dashed',lw=0.7,color='k')
    ax.axvline(x=50,ls='dashed',lw=0.7,color='k')
    ax.axvline(x=60,ls='dashed',lw=0.7,color='k')
    ax.set_xlim(-5,65)
    
    ax.set_xticks([0,10,20,30,40,50,60]) # 设置刻度
    ax.set_xticklabels(X_lable)
    ax.set_title(lead_index[i])
    # for j in t:
    #     ax.axvline(x=j,alpha=0.1,color=colors[i,j]) #第i导联 第j个时间点的importance value

In [ ]:
lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
X_lable = ['0','T','P','Q','R','S','T','P','']

x_index = np.arange(0,12)
fig, axs = plt.subplots(nrows=3, ncols=4, sharex=True,sharey=True,figsize=(8,10), constrained_layout=True,dpi =100)
ecg_data,_ = test_dataset.__getitem__(index)

cmap = 'OrRd' 
colors = ECGplot.color_map(important_values, cmap)#相当于归一化

for i,ax in enumerate(axs.flat):  # type: ignore
    plot_y = np.array(ECG_values[i]*5000.)#缩放
    t = np.arange(0,segement_length)
    ECGplot.plot_ECG_line(fig,ax,x = t,y= plot_y,y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i])
    for j in t:
        ax.axvline(x=j,alpha=0.1,color=colors[i,j]) #第i导联 第j个时间点的importance value

---

In [ ]:
index = 0
segement_number = 10
important_values = np.zeros([6,segement_number])
Occlusion_root = save_root+'/Occlusion/'

for index in range(test_dataset.__len__()):#
    important_value = np.zeros([6,segement_number])
    inputs,labels = test_dataset.__getitem__(index)
    if(labels == 0) :continue
    info =test_dataset.infos.iloc[index]
    ECGfile_name = info['ECGFilename']
    layervalue_file = Occlusion_root  + ECGfile_name +'.npy'
    waves_location_file_root = '/workspace/data/Preprocess_HTN/Waves_2023/'
    if(not os.path.exists(layervalue_file)):
        print(layervalue_file," not exit")
        continue
    waves = pd.read_csv(waves_location_file_root+ECGfile_name+'.csv',index_col=0)
    if(waves.isnull().any().any()):
        continue
    layervalue = np.load(layervalue_file)

    for head_peak in range(1,len(waves.columns)-1):
        if(waves.loc['P'][head_peak] == waves.iloc[:,head_peak].min()): #P波不是第一个识别出来的波形
            p_index = int(waves.loc['P'][head_peak])
            q_index = int(waves.loc['Q'][head_peak])
            r_index = int(waves.loc['R'][head_peak])
            s_index = int(waves.loc['S'][head_peak])
            t_index = int(waves.loc['T'][head_peak])
            t_index_befort = int(waves.loc['T'][head_peak-1])
            if(((waves.loc['P'][head_peak+1]==None)) or((waves.loc['P'][head_peak+1] < t_index))):
                break 
            else:
                p_index_next = waves.loc['P'][head_peak+1] 
            important_value[0] = important_value[0]+ layervalue_segement_count(t_index_befort,p_index,layervalue,segement_number)
            important_value[1] = important_value[1]+ layervalue_segement_count(p_index,q_index,layervalue,segement_number)   
            important_value[2] = important_value[2]+ layervalue_segement_count(q_index,r_index,layervalue,segement_number) 
            important_value[3] = important_value[3]+layervalue_segement_count(r_index,s_index,layervalue,segement_number)
            important_value[4] = important_value[4]+ layervalue_segement_count(s_index,t_index,layervalue,segement_number) 
            important_value[5] = important_value[5]+ layervalue_segement_count(t_index,p_index_next,layervalue,segement_number) 
        else:
            break
    important_value = max_min(important_value)
    important_values = important_values + important_value
X_lable = ['0','T','P','Q','R','S','T','P','']
fig,ax = plt.subplots(1,1)
ax.plot(np.arange(important_values.flatten().__len__()), max_min(important_values.flatten()), lw=1,color='k')
ax.axvline(x=0,ls='dashed',lw=0.7,color='r')
ax.axvline(x=10,ls='dashed',lw=0.7,color='g')
ax.axvline(x=20,ls='dashed',lw=0.7,color='b')
ax.axvline(x=30,ls='dashed',lw=0.7,color='c')
ax.axvline(x=40,ls='dashed',lw=0.7,color='m')
ax.axvline(x=50,ls='dashed',lw=0.7,color='r')
ax.axvline(x=60,ls='dashed',lw=0.7,color='g')
ax.set_xlim(-5,65)
ax.set_xticklabels(X_lable)

In [ ]:
index = 0
segement_number = 10
important_values = np.zeros([6,segement_number])
Occlusion_root = save_root+'/Occlusion/'

for index in range(test_dataset.__len__()):#
    important_value = np.zeros([6,segement_number])
    inputs,labels = test_dataset.__getitem__(index)
    if(labels == 1) :continue
    info =test_dataset.infos.iloc[index]
    ECGfile_name = info['ECGFilename']
    layervalue_file = Occlusion_root  + ECGfile_name +'.npy'
    waves_location_file_root = '/workspace/data/Preprocess_HTN/Waves_2023/'
    if(not os.path.exists(layervalue_file)):
        print(layervalue_file," not exit")
        continue
    waves = pd.read_csv(waves_location_file_root+ECGfile_name+'.csv',index_col=0)
    if(waves.isnull().any().any()):
        continue
    layervalue = np.load(layervalue_file)

    for head_peak in range(1,len(waves.columns)-1):
        if(waves.loc['P'][head_peak] == waves.iloc[:,head_peak].min()): #P波不是第一个识别出来的波形
            p_index = int(waves.loc['P'][head_peak])
            q_index = int(waves.loc['Q'][head_peak])
            r_index = int(waves.loc['R'][head_peak])
            s_index = int(waves.loc['S'][head_peak])
            t_index = int(waves.loc['T'][head_peak])
            t_index_befort = int(waves.loc['T'][head_peak-1])
            if(((waves.loc['P'][head_peak+1]==None)) or((waves.loc['P'][head_peak+1] < t_index))):
                break
            else:
                p_index_next = waves.loc['P'][head_peak+1] 
            important_value[0] = important_value[0]+ layervalue_segement_count(t_index_befort,p_index,layervalue,segement_number)
            important_value[1] = important_value[1]+ layervalue_segement_count(p_index,q_index,layervalue,segement_number)   
            important_value[2] = important_value[2]+ layervalue_segement_count(q_index,r_index,layervalue,segement_number) 
            important_value[3] = important_value[3]+layervalue_segement_count(r_index,s_index,layervalue,segement_number)
            important_value[4] = important_value[4]+ layervalue_segement_count(s_index,t_index,layervalue,segement_number) 
            important_value[5] = important_value[5]+ layervalue_segement_count(t_index,p_index_next,layervalue,segement_number) 
        else:
            break
    important_value = max_min(important_value)
    important_values = important_values + important_value
X_lable = ['0','T','P','Q','R','S','T','P','']
fig,ax = plt.subplots(1,1)
ax.plot(np.arange(important_values.flatten().__len__()), max_min(important_values.flatten()), lw=1,color='k')
ax.axvline(x=0,ls='dashed',lw=0.7,color='r')
ax.axvline(x=10,ls='dashed',lw=0.7,color='g')
ax.axvline(x=20,ls='dashed',lw=0.7,color='b')
ax.axvline(x=30,ls='dashed',lw=0.7,color='c')
ax.axvline(x=40,ls='dashed',lw=0.7,color='m')
ax.axvline(x=50,ls='dashed',lw=0.7,color='r')
ax.axvline(x=60,ls='dashed',lw=0.7,color='g')
ax.set_xlim(-5,65)
ax.set_xticklabels(X_lable)

----------------------------

GRU：2 ATT  batchsize：64

In [ ]:
np.array([0.3512900727135794, 0.3734808053289141, 0.3555836294378553, 0.3445756807923317, 0.34377362579107285]).mean() #logs/20230212_134040/log.log

In [ ]:
np.array([0.8434311224489796, 0.8252060439560439, 0.852782392026578, 0.861328125, 0.864453125] ).mean() 

-----------------

3 layer GRU  batchsize = 64

In [ ]:
np.array([0.34137497203690664, 0.419583329132625, 0.2343021673815591, 0.3741154298186302, 0.2929470892995596]).mean()  #/logs/20230211_171822/log.log    3 layer GRU  batchsize = 64

In [ ]:
np.array([0.8499453352769679, 0.823489010989011, 0.9052637043189369, 0.8460582386363636, 0.88359375]).mean()  

In [ ]:
np.array([0.9523809523809523,0.8523809523809524,0.9790575916230366,0.8393574297188755,0.8310344827586207]).mean()  #precision

In [ ]:
np.array([0.7339449541284404,0.7955555555555556,0.8385650224215246, 0.8565573770491803,0.8763636363636363]).mean()  #recall

------------

2 layer GRU  batchsize = 128

In [ ]:
np.array([0.32283543795347214, 0.44184040278196335, 0.29647157341241837, 0.4000786542892456, 0.342408150434494]).mean() # vlogs/20230211_072050/log.log   2 layer GRU  batchsize = 128

In [ ]:
np.array([0.8651147959183674, 0.794921875, 0.8811319040697674, 0.8361458333333334, 0.8776041666666666] ).mean() 

----------

 2 layer GRU  batchsize = 64

In [ ]:
np.array( [0.3219981959887913, 0.3934356187071119, 0.28491889366081785, 0.33339295722544193, 0.3012485932558775]).mean()  # logs/20230212_031236/log.log

In [ ]:
np.array([0.8693513119533528, 0.835679945054945, 0.8918708471760797, 0.869140625, 0.88828125] ).mean() 

In [ ]:
np.array([0.9497206703910615, 0.8768472906403941,0.9538461538461539,0.9447236180904522,0.8988326848249028]).mean()  #precision

In [ ]:
np.array( [0.7798165137614679,0.7911111111111111, 0.8340807174887892, 0.7704918032786885,0.8884615384615384]).mean()  #recall

GradientShap

In [ ]:
NET = [Net.MLBFNet_GUR_o(True,True,True,2,Dropout_rate=0.3) ] # type: ignore
testmodel = NET[0].to(DEVICE)
Models_path = '/workspace/data/Interpretable_HTN/model/20230212_031236/20230212_031236/parameter_EarlyStoping_4.pt'
save_root = Models_path[:-3]+'/'
layervalue_root = save_root+'/layervalue/'
testmodel.load_state_dict(torch.load(Models_path))

In [ ]:
def predict(data,testmodel):
    testmodel.eval()
    output = testmodel(data)
    porbs = torch.nn.functional.softmax(output)
    return porbs

In [ ]:
inputs,labels = test_dataset.__getitem__(index)
info =test_dataset.infos.iloc[index]
ECGfile_name = info['ECGFilename']
testmodel.eval()
inputs = inputs.unsqueeze(0)
inputs = inputs.to(DEVICE)
labels = labels.to(DEVICE)  
predict(inputs,testmodel)

In [ ]:
labels

In [ ]:
from captum.attr import GradientShap
from captum.attr import visualization as viz

In [ ]:
gradient_shap = GradientShap(testmodel)

In [ ]:
rand_img_dist = torch.cat([inputs * 0, inputs * 1])

In [ ]:
rand_img_dist.shape

In [ ]:
attributions_gs = gradient_shap.attribute(inputs,
                                          n_samples=50,
                                          stdevs=0.0001,
                                          baselines=rand_img_dist,
                                          target=labels)


In [ ]:
attributions_gs.shape

遮挡可解释性分析

In [ ]:
from captum.attr import Occlusion
from captum.attr import visualization as viz

In [ ]:
occlusion = Occlusion(testmodel)

In [ ]:
save_root = Models_path[:-3]+'/'
Occlusion_root = save_root+'/Occlusion/'
if(not(os.path.exists(save_root))): os.mkdir(save_root)
if(not(os.path.exists(Occlusion_root))): os.mkdir(Occlusion_root)
lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
x_index = np.arange(0,12)
# fig, axs = plt.subplots(nrows=4, ncols=3, sharex=True,sharey=True,figsize=(45,25), constrained_layout=True,dpi =100)
for index in tqdm(range(test_dataset.__len__())):#test_dataset.__len__()
    
    inputs,labels = test_dataset.__getitem__(index)
    info =test_dataset.infos.iloc[index]
    inputs = inputs.unsqueeze(0)
    ECGfile_name = info['ECGFilename']
    attributions_occ = occlusion.attribute(inputs,
                                        strides = (1, 10), # 遮挡滑动移动步长
                                        target=labels, # 目标类别
                                        sliding_window_shapes=(12, 20), # 遮挡滑块尺寸
                                        baselines=0)
    Occlusion_vlue = attributions_occ.squeeze(0).numpy()
    np.save(Occlusion_root+ECGfile_name,Occlusion_vlue)

In [ ]:
layer2d_vlue = attributions_occ.squeeze(0).numpy()
lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
x_index = np.arange(0,12)
fig, axs = plt.subplots(nrows=4, ncols=3, sharex=True,sharey=True,figsize=(45,25), constrained_layout=True,dpi =100)
ecg_data,_ = test_dataset.__getitem__(index)

cmap = 'OrRd' 
colors = ECGplot.color_map(layer2d_vlue, cmap)#相当于归一化

for i,ax in enumerate(axs.flat):  # type: ignore
    plot_y = np.array(ecg_data[i]*5000.)#缩放
    t = np.arange(0,5000)
    ECGplot.plot_ECG_line(fig,ax,x = t,y= plot_y,y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i])
    for j in t:
        ax.axvline(x=j,alpha=0.1,color=colors[i,j]) #第i导联 第j个时间点的importance value